# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [18]:
data_df = pd.read_csv("../output_data/cities.csv")
data_df



,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05
...,...,...,...,...,...,...,...,...,...,...
543,543,yabrud,0,SY,1558378840,32,33.97,36.66,80.60,8.05
544,544,paraiso,5,MX,1558378898,4,24.01,-104.61,84.20,16.11
545,545,veraval,0,FR,1558378898,62,49.65,0.71,61.00,8.05
546,546,novyy urgal,100,RU,1558378899,93,51.07,132.56,49.74,2.68


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [19]:
gmaps.configure(api_key=g_key)

In [20]:
locations = data_df[["Lat", "Lng"]].astype(float)
locations

,Lat,Lng
0,68.05,39.51
1,-20.41,57.70
2,77.48,-69.36
3,40.71,112.04
4,-33.93,18.42
...,...,...
543,33.97,36.66
544,24.01,-104.61
545,49.65,0.71
546,51.07,132.56


In [21]:
Hum_df = data_df["Humidity"]
Hum_df

0      72
1      74
2      73
3      39
4      76
       ..
543    32
544     4
545    62
546    93
547    64
Name: Humidity, Length: 548, dtype: int64

In [11]:
# Create a Humidity Heatmap layer
fig = gmaps.figure()

humidity_layer = gmaps.heatmap_layer(locations, weights=Hum_df, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 5)

fig.add_layer(humidity_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [22]:
data_weather_cri_df= data_df.loc[(data_df['Max Temp']<80) & (data_df['Max Temp']>70) & (data_df['Wind Speed']< 10) & (data_df['Cloudiness'] == 0)]

# drop empty dataframe
weather_df = data_weather_cri_df.dropna()
weather_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
70,70,ponta do sol,0,BR,1558378767,57,-20.63,-46.00,73.68,5.32
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
138,138,morondava,0,MG,1558378780,69,-20.30,44.28,79.08,9.06
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59
176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93
297,297,mogok,0,MM,1558378844,51,22.92,96.51,72.24,2.77
444,444,birjand,0,IR,1558378880,26,32.86,59.22,71.60,9.17


In [31]:
hotel_df = weather_df[['City', 'Country', 'Lat', 'Lng']].copy()
hotel_df['Hotel Name'] = ""

hotel_df

,City,Country,Lat,Lng,Hotel Name
70,ponta do sol,BR,-20.63,-46.00,
88,cidreira,BR,-30.17,-50.22,
138,morondava,MG,-20.30,44.28,
139,vaini,IN,15.34,74.49,
176,nador,MA,35.17,-2.93,
297,mogok,MM,22.92,96.51,
444,birjand,IR,32.86,59.22,


In [46]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}


for index, row in hotel_df.iterrows():
    Latitude = row['Lat']
    Longitude = row['Lng']
    params['location']= f"{Latitude}, {Longitude}"
    #print (row["Country"], row["City"])
    response =requests.get(base_url, params=params)
    
    response_data = response.json()
    HotelName = response_data['results'][0]['name']
    
    hotel_df.loc[index, "Hotel Name"]= HotelName

hotel_df

    
    

,City,Country,Lat,Lng,Hotel Name
70,ponta do sol,BR,-20.63,-46.00,Hotel Escarpas do Lago
88,cidreira,BR,-30.17,-50.22,Hotel Castelo
138,morondava,MG,-20.30,44.28,La Case Bambou
139,vaini,IN,15.34,74.49,Dandeli Lake County
176,nador,MA,35.17,-2.93,Hôtel La Giralda
297,mogok,MM,22.92,96.51,MOGOK HILL HOTEL
444,birjand,IR,32.86,59.22,Birjand Tourist Hotel


In [38]:
from pprint import pprint
pprint(response_data)

{'html_attributions': [],
 'next_page_token': 'Aap_uEBPvzTEaQoX9Sj-KNsP_1nj8Fp6Os2DPXkzd4JNlI3xfXX_kZdnLJvd5adGyaYG8MLt9t9Qz9iZYu508egYyOqeOooT6K_RpE7gfD-vH3NpnRQ_XB2kX3hyBOBD8vqRWJGTzf7XvfZQ130Ni6As3LDlb7m2UoKNmtKMdQQWcd8x7w6IVfK0nC76nHsBWQE6vME6DiuoxyqqI3GJhEy6f60DeMXxJYsfpzDXKJ_bB8DDbBMJTRNLRRf31Xss4qhDnLpwQcZjL_w9cnyHuz04VYW_Tm50EXq8xGNiukW35KPGJlRzj0GbrTaWzUCTEjvw7_YZDuXsuZlFbMosv4pGFZWUiHzcXsigMKcs73c-W7KK9lKeQDefo2uJkCyJHTGyUpPaINf_wH3o_hSd0wWr8COd2pbry8ylj_nfv9POLUMK-8GqISf5Ukna',
 'results': [{'business_status': 'OPERATIONAL',
              'geometry': {'location': {'lat': 32.8713752, 'lng': 59.2307798},
                           'viewport': {'northeast': {'lat': 32.8728181302915,
                                                      'lng': 59.2321445802915},
                                        'southwest': {'lat': 32.8701201697085,
                                                      'lng': 59.22944661970849}}},
              'icon': 'https://maps.gstatic.com/mapfiles/p

In [44]:
response_data['results'][0]['name']

'Birjand Tourist Hotel'

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [49]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [50]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content =hotel_info)
fig.add_layer(marker_layer)


# Display figure
fig


Figure(layout=FigureLayout(height='420px'))